In [4]:
!pip install datasets
!pip install transformers

  Using cached https://files.pythonhosted.org/packages/94/f8/ff7cd6e3b400b33dcbbfd31c6c1481678a2b2f669f521ad20053009a9aa3/datasets-1.7.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/52/816d1a3a599176057bf29dfacb1f8fadb61d35fbd96cb1bab4aaa7df83c0/fsspec-2021.5.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7d/4f/0a862cad26aa2ed7a7cd87178cbbfa824fc1383e472d63596a0d018374e7/xxhash-2.0.2-cp37-cp37m-manylinux2010_x86_64.whl


In [5]:
import transformers
import datasets
import torch
from datasets import load_dataset

class NLITRReader(torch.utils.data.Dataset):
  def __init__(self, dataset_name, split_name, max_example_num=-1):
    self.dataset = load_dataset('nli_tr', dataset_name)
    self.split_name = split_name
    self.max_example_num = max_example_num

  def read(self):
      count = 0
      for example in self.dataset[self.split_name]:
          if example['label'] == -1: # skip examples having no gold value.
              continue
          count += 1
          if self.max_example_num > 0 and count >= self.max_example_num:
             break
          yield example

In [6]:
import torch
class NLITRDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [7]:
snli_tr_dataset = load_dataset('nli_tr', 'snli_tr')

Dataset nli_tr downloaded and prepared to /root/.cache/huggingface/datasets/nli_tr/snli_tr/1.0.0/c2ddd0c0a70caddac6a81c2dae5ca7939f00060d517d08f1983927818dba6521. Subsequent calls will reuse this data.


In [8]:
print(snli_tr_dataset.shape)

{'train': (550152, 4), 'validation': (10000, 4), 'test': (10000, 4)}


In [9]:
train_data = snli_tr_dataset['train']
valid_data = snli_tr_dataset['validation']
test_data = snli_tr_dataset['test']

In [10]:
import pandas as pd
train_data_df = pd.DataFrame(train_data)

In [11]:
train_data_df

,hypothesis,idx,label,premise
0,Bir kişi atını yarışma için eğitiyor.,0,1,"Attaki bir kişi, bozuk bir uçağın üzerinden at..."
1,Bir kişi bir lokantada omlet sipariş ediyor.,1,2,"Attaki bir kişi, bozuk bir uçağın üzerinden at..."
2,"Bir kişi açık havada, at üzerinde.",2,0,"Attaki bir kişi, bozuk bir uçağın üzerinden at..."
3,Ailelerine gülümsüyorlar.,3,1,Fotoğraf makinesinde gülümseyen ve sallayan ço...
4,Burada çocuklar var.,4,0,Fotoğraf makinesinde gülümseyen ve sallayan ço...
...,...,...,...,...
550147,dört çocuk'temiz ayaklar'için ödül kazandı,550147,2,Dört kirli ve çıplak ayaklı çocuk.
550148,"Dört evsiz çocuğun ayakkabısı çalınmış, bu yüz...",550148,1,Dört kirli ve çıplak ayaklı çocuk.
550149,Tulum giyen bir adam sörf yarışmasında yarışıyor.,550149,1,Bir adam güzel mavi suda bir tulum içinde sörf...
550150,Takım elbiseli bir adam yönetim kurulu toplant...,550150,2,Bir adam güzel mavi suda bir tulum içinde sörf...


In [12]:
del train_data_df['idx']
train_data_df

,hypothesis,label,premise
0,Bir kişi atını yarışma için eğitiyor.,1,"Attaki bir kişi, bozuk bir uçağın üzerinden at..."
1,Bir kişi bir lokantada omlet sipariş ediyor.,2,"Attaki bir kişi, bozuk bir uçağın üzerinden at..."
2,"Bir kişi açık havada, at üzerinde.",0,"Attaki bir kişi, bozuk bir uçağın üzerinden at..."
3,Ailelerine gülümsüyorlar.,1,Fotoğraf makinesinde gülümseyen ve sallayan ço...
4,Burada çocuklar var.,0,Fotoğraf makinesinde gülümseyen ve sallayan ço...
...,...,...,...
550147,dört çocuk'temiz ayaklar'için ödül kazandı,2,Dört kirli ve çıplak ayaklı çocuk.
550148,"Dört evsiz çocuğun ayakkabısı çalınmış, bu yüz...",1,Dört kirli ve çıplak ayaklı çocuk.
550149,Tulum giyen bir adam sörf yarışmasında yarışıyor.,1,Bir adam güzel mavi suda bir tulum içinde sörf...
550150,Takım elbiseli bir adam yönetim kurulu toplant...,2,Bir adam güzel mavi suda bir tulum içinde sörf...


In [13]:
columns_titles = ["label","premise","hypothesis"]

df_reorder=train_data_df.reindex(columns=columns_titles)
df_reorder

,label,premise,hypothesis
0,1,"Attaki bir kişi, bozuk bir uçağın üzerinden at...",Bir kişi atını yarışma için eğitiyor.
1,2,"Attaki bir kişi, bozuk bir uçağın üzerinden at...",Bir kişi bir lokantada omlet sipariş ediyor.
2,0,"Attaki bir kişi, bozuk bir uçağın üzerinden at...","Bir kişi açık havada, at üzerinde."
3,1,Fotoğraf makinesinde gülümseyen ve sallayan ço...,Ailelerine gülümsüyorlar.
4,0,Fotoğraf makinesinde gülümseyen ve sallayan ço...,Burada çocuklar var.
...,...,...,...
550147,2,Dört kirli ve çıplak ayaklı çocuk.,dört çocuk'temiz ayaklar'için ödül kazandı
550148,1,Dört kirli ve çıplak ayaklı çocuk.,"Dört evsiz çocuğun ayakkabısı çalınmış, bu yüz..."
550149,1,Bir adam güzel mavi suda bir tulum içinde sörf...,Tulum giyen bir adam sörf yarışmasında yarışıyor.
550150,2,Bir adam güzel mavi suda bir tulum içinde sörf...,Takım elbiseli bir adam yönetim kurulu toplant...


In [14]:
df_reorder.columns = ['gold_label', 'sentence1', 'sentence2']
df_reorder

,gold_label,sentence1,sentence2
0,1,"Attaki bir kişi, bozuk bir uçağın üzerinden at...",Bir kişi atını yarışma için eğitiyor.
1,2,"Attaki bir kişi, bozuk bir uçağın üzerinden at...",Bir kişi bir lokantada omlet sipariş ediyor.
2,0,"Attaki bir kişi, bozuk bir uçağın üzerinden at...","Bir kişi açık havada, at üzerinde."
3,1,Fotoğraf makinesinde gülümseyen ve sallayan ço...,Ailelerine gülümsüyorlar.
4,0,Fotoğraf makinesinde gülümseyen ve sallayan ço...,Burada çocuklar var.
...,...,...,...
550147,2,Dört kirli ve çıplak ayaklı çocuk.,dört çocuk'temiz ayaklar'için ödül kazandı
550148,1,Dört kirli ve çıplak ayaklı çocuk.,"Dört evsiz çocuğun ayakkabısı çalınmış, bu yüz..."
550149,1,Bir adam güzel mavi suda bir tulum içinde sörf...,Tulum giyen bir adam sörf yarışmasında yarışıyor.
550150,2,Bir adam güzel mavi suda bir tulum içinde sörf...,Takım elbiseli bir adam yönetim kurulu toplant...


In [ ]:
# import numpy as np
# np.savetxt(r'snli_tr_train.txt', df_reorder.values, fmt='%s')

In [29]:
import os
pd.DataFrame(df_reorder).to_csv(os.getcwd()+'/train.csv', sep='\t', index=False, header=True)
#df_reorder.to_csv(r'train.csv', sep='\t', index=False, header=True)

In [31]:
pd.DataFrame(df_reorder).to_csv(os.getcwd()+'/train_index.csv', sep='\t', index=True, header=True)

In [17]:
import os
pd.DataFrame(df_reorder).to_csv(os.getcwd()+'/train_index.tsv', sep='\t', index=True, header=True)

In [18]:
#VALIDATION
import pandas as pd
valid_data_df = pd.DataFrame(valid_data)
valid_data_df

,hypothesis,idx,label,premise
0,Kız kardeşler sadece öğle yemeği yedikten sonr...,0,1,Paketlere gitmek için tutunurken iki kadın kuc...
1,İki kadın paket tutuyor.,1,0,Paketlere gitmek için tutunurken iki kadın kuc...
2,Adamlar bir şarküterinin dışında kavga ediyorlar.,2,2,Paketlere gitmek için tutunurken iki kadın kuc...
3,Numaralı formalı iki çocuk ellerini yıkar.,3,0,"Mavi formalı iki küçük çocuk, biri 9 numara ve..."
4,Bir beyzbol maçında iki çocuk ellerini yıkar.,4,1,"Mavi formalı iki küçük çocuk, biri 9 numara ve..."
...,...,...,...,...
9995,Kız ahşaptan yapılmış bir oyma at üzerinde otu...,9995,-1,Pembe bir ceket giyen küçük bir kız bir atlıka...
9996,Kız ışık hızında hareket ediyor.,9996,2,Pembe bir ceket giyen küçük bir kız bir atlıka...
9997,Bir su çeşmesinde insanlar,9997,0,Örgülerinde mavi ve pembe kurdeleler olan genç...
9998,Genç bir kız bir kazak örer,9998,2,Örgülerinde mavi ve pembe kurdeleler olan genç...


In [19]:
del valid_data_df['idx']
valid_data_df

,hypothesis,label,premise
0,Kız kardeşler sadece öğle yemeği yedikten sonr...,1,Paketlere gitmek için tutunurken iki kadın kuc...
1,İki kadın paket tutuyor.,0,Paketlere gitmek için tutunurken iki kadın kuc...
2,Adamlar bir şarküterinin dışında kavga ediyorlar.,2,Paketlere gitmek için tutunurken iki kadın kuc...
3,Numaralı formalı iki çocuk ellerini yıkar.,0,"Mavi formalı iki küçük çocuk, biri 9 numara ve..."
4,Bir beyzbol maçında iki çocuk ellerini yıkar.,1,"Mavi formalı iki küçük çocuk, biri 9 numara ve..."
...,...,...,...
9995,Kız ahşaptan yapılmış bir oyma at üzerinde otu...,-1,Pembe bir ceket giyen küçük bir kız bir atlıka...
9996,Kız ışık hızında hareket ediyor.,2,Pembe bir ceket giyen küçük bir kız bir atlıka...
9997,Bir su çeşmesinde insanlar,0,Örgülerinde mavi ve pembe kurdeleler olan genç...
9998,Genç bir kız bir kazak örer,2,Örgülerinde mavi ve pembe kurdeleler olan genç...


In [20]:
columns_titles = ["label","premise","hypothesis"]

df_reorder_valid=valid_data_df.reindex(columns=columns_titles)
df_reorder_valid

,label,premise,hypothesis
0,1,Paketlere gitmek için tutunurken iki kadın kuc...,Kız kardeşler sadece öğle yemeği yedikten sonr...
1,0,Paketlere gitmek için tutunurken iki kadın kuc...,İki kadın paket tutuyor.
2,2,Paketlere gitmek için tutunurken iki kadın kuc...,Adamlar bir şarküterinin dışında kavga ediyorlar.
3,0,"Mavi formalı iki küçük çocuk, biri 9 numara ve...",Numaralı formalı iki çocuk ellerini yıkar.
4,1,"Mavi formalı iki küçük çocuk, biri 9 numara ve...",Bir beyzbol maçında iki çocuk ellerini yıkar.
...,...,...,...
9995,-1,Pembe bir ceket giyen küçük bir kız bir atlıka...,Kız ahşaptan yapılmış bir oyma at üzerinde otu...
9996,2,Pembe bir ceket giyen küçük bir kız bir atlıka...,Kız ışık hızında hareket ediyor.
9997,0,Örgülerinde mavi ve pembe kurdeleler olan genç...,Bir su çeşmesinde insanlar
9998,2,Örgülerinde mavi ve pembe kurdeleler olan genç...,Genç bir kız bir kazak örer


In [21]:
df_reorder_valid.columns = ['gold_label', 'sentence1', 'sentence2']
df_reorder_valid

,gold_label,sentence1,sentence2
0,1,Paketlere gitmek için tutunurken iki kadın kuc...,Kız kardeşler sadece öğle yemeği yedikten sonr...
1,0,Paketlere gitmek için tutunurken iki kadın kuc...,İki kadın paket tutuyor.
2,2,Paketlere gitmek için tutunurken iki kadın kuc...,Adamlar bir şarküterinin dışında kavga ediyorlar.
3,0,"Mavi formalı iki küçük çocuk, biri 9 numara ve...",Numaralı formalı iki çocuk ellerini yıkar.
4,1,"Mavi formalı iki küçük çocuk, biri 9 numara ve...",Bir beyzbol maçında iki çocuk ellerini yıkar.
...,...,...,...
9995,-1,Pembe bir ceket giyen küçük bir kız bir atlıka...,Kız ahşaptan yapılmış bir oyma at üzerinde otu...
9996,2,Pembe bir ceket giyen küçük bir kız bir atlıka...,Kız ışık hızında hareket ediyor.
9997,0,Örgülerinde mavi ve pembe kurdeleler olan genç...,Bir su çeşmesinde insanlar
9998,2,Örgülerinde mavi ve pembe kurdeleler olan genç...,Genç bir kız bir kazak örer


In [40]:
import os
pd.DataFrame(df_reorder_valid).to_csv(os.getcwd()+'/valid_noIndex.csv', sep='\t', index=False, header=True)

In [23]:
import os
pd.DataFrame(df_reorder_valid).to_csv(os.getcwd()+'/valid_index.tsv', sep='\t', index=True, header=True)

In [30]:
import os
pd.DataFrame(df_reorder_valid).to_csv(os.getcwd()+'/valid_noIndex.tsv', sep='\t', index=False, header=True)

In [43]:
pd.DataFrame(df_reorder_valid).to_csv(os.getcwd()+'/valid_index.csv', sep='\t', index=True, header=True)

In [31]:
#TEST
import pandas as pd
test_data_df = pd.DataFrame(test_data)
test_data_df

,hypothesis,idx,label,premise
0,Kilisenin tavanında çatlaklar var.,0,1,"Bu kilise korosu, kilisedeki kitaptan neşeli ş..."
1,Kilise şarkıyla dolu.,1,0,"Bu kilise korosu, kilisedeki kitaptan neşeli ş..."
2,Beyzbol maçında şarkı söyleyen bir koro.,2,2,"Bu kilise korosu, kilisedeki kitaptan neşeli ş..."
3,Kadın genç.,3,1,"Yeşil başörtüsü, mavi gömlekli ve çok büyük bi..."
4,Kadın çok mutlu.,4,0,"Yeşil başörtüsü, mavi gömlekli ve çok büyük bi..."
...,...,...,...,...
9995,İki kadın gözleri kapalı duruyor.,9995,2,İki kadın birlikte bir şeyi gözlemliyor.
9996,İki kız bir şeye bakıyor.,9996,0,İki kadın birlikte bir şeyi gözlemliyor.
9997,Bir adam uçurtma uçuruyor.,9997,2,Siyah deri ceketli bir adam ve elinde bir kita...
9998,Bir adam sınıfta konuşuyor.,9998,0,Siyah deri ceketli bir adam ve elinde bir kita...


In [32]:
del test_data_df['idx']
test_data_df

,hypothesis,label,premise
0,Kilisenin tavanında çatlaklar var.,1,"Bu kilise korosu, kilisedeki kitaptan neşeli ş..."
1,Kilise şarkıyla dolu.,0,"Bu kilise korosu, kilisedeki kitaptan neşeli ş..."
2,Beyzbol maçında şarkı söyleyen bir koro.,2,"Bu kilise korosu, kilisedeki kitaptan neşeli ş..."
3,Kadın genç.,1,"Yeşil başörtüsü, mavi gömlekli ve çok büyük bi..."
4,Kadın çok mutlu.,0,"Yeşil başörtüsü, mavi gömlekli ve çok büyük bi..."
...,...,...,...
9995,İki kadın gözleri kapalı duruyor.,2,İki kadın birlikte bir şeyi gözlemliyor.
9996,İki kız bir şeye bakıyor.,0,İki kadın birlikte bir şeyi gözlemliyor.
9997,Bir adam uçurtma uçuruyor.,2,Siyah deri ceketli bir adam ve elinde bir kita...
9998,Bir adam sınıfta konuşuyor.,0,Siyah deri ceketli bir adam ve elinde bir kita...


In [33]:
columns_titles = ["label","premise","hypothesis"]

df_reorder_test=test_data_df.reindex(columns=columns_titles)
df_reorder_test

,label,premise,hypothesis
0,1,"Bu kilise korosu, kilisedeki kitaptan neşeli ş...",Kilisenin tavanında çatlaklar var.
1,0,"Bu kilise korosu, kilisedeki kitaptan neşeli ş...",Kilise şarkıyla dolu.
2,2,"Bu kilise korosu, kilisedeki kitaptan neşeli ş...",Beyzbol maçında şarkı söyleyen bir koro.
3,1,"Yeşil başörtüsü, mavi gömlekli ve çok büyük bi...",Kadın genç.
4,0,"Yeşil başörtüsü, mavi gömlekli ve çok büyük bi...",Kadın çok mutlu.
...,...,...,...
9995,2,İki kadın birlikte bir şeyi gözlemliyor.,İki kadın gözleri kapalı duruyor.
9996,0,İki kadın birlikte bir şeyi gözlemliyor.,İki kız bir şeye bakıyor.
9997,2,Siyah deri ceketli bir adam ve elinde bir kita...,Bir adam uçurtma uçuruyor.
9998,0,Siyah deri ceketli bir adam ve elinde bir kita...,Bir adam sınıfta konuşuyor.


In [34]:
df_reorder_test.columns = ['gold_label', 'sentence1', 'sentence2']
df_reorder_test

,gold_label,sentence1,sentence2
0,1,"Bu kilise korosu, kilisedeki kitaptan neşeli ş...",Kilisenin tavanında çatlaklar var.
1,0,"Bu kilise korosu, kilisedeki kitaptan neşeli ş...",Kilise şarkıyla dolu.
2,2,"Bu kilise korosu, kilisedeki kitaptan neşeli ş...",Beyzbol maçında şarkı söyleyen bir koro.
3,1,"Yeşil başörtüsü, mavi gömlekli ve çok büyük bi...",Kadın genç.
4,0,"Yeşil başörtüsü, mavi gömlekli ve çok büyük bi...",Kadın çok mutlu.
...,...,...,...
9995,2,İki kadın birlikte bir şeyi gözlemliyor.,İki kadın gözleri kapalı duruyor.
9996,0,İki kadın birlikte bir şeyi gözlemliyor.,İki kız bir şeye bakıyor.
9997,2,Siyah deri ceketli bir adam ve elinde bir kita...,Bir adam uçurtma uçuruyor.
9998,0,Siyah deri ceketli bir adam ve elinde bir kita...,Bir adam sınıfta konuşuyor.


In [49]:
pd.DataFrame(df_reorder_test).to_csv(os.getcwd()+'/test_noIndex.csv', sep='\t', index=False, header=True)

In [51]:
pd.DataFrame(df_reorder_test).to_csv(os.getcwd()+'/test_index.csv', sep='\t', index=True, header=True)

In [37]:
pd.DataFrame(df_reorder_test).to_csv(os.getcwd()+'/test_index.tsv', sep='\t', index=True, header=True)

In [39]:
import os
pd.DataFrame(df_reorder).to_csv(os.getcwd()+'/train_noIndex.tsv', sep='\t', index=False, header=True)

In [36]:
import os
pd.DataFrame(df_reorder_test).to_csv(os.getcwd()+'/test_noIndex.tsv', sep='\t', index=False, header=True)